In [1]:
import cbpro
import datetime
import pandas as pd
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import LinearAxis, Range1d, NumeralTickFormatter
output_notebook()

Loading BokehJS ...

In [2]:
def get_product_data(product='BTC-USD', granularity=86400):
    public_client = cbpro.PublicClient()
    return public_client.get_product_historic_rates(product, granularity=granularity)

def axis_range(values, padding=0.1):
    delta = max(values) - min(values)
    low = min(values) - padding*delta
    high = max(values) + padding*delta
    return [low, high]

def make_rate_plot(times, values, volumes, crypto_label='BTC', fiat_label='USD'):
    title = crypto_label + ' Price'
    p = figure(x_axis_type="datetime", title="BTC Price", plot_height=350, plot_width=800)
    p.xgrid.grid_line_color=None
    p.ygrid.grid_line_alpha=0.5
    p.xaxis.axis_label = 'Time'

    # Set up the first y-axis
    p.yaxis.axis_label = 'Price (' + fiat_label + ')'
    p.y_range = Range1d(*axis_range(values))
    p.yaxis.formatter = NumeralTickFormatter(format="$0,0.")
    p.line(times, values, line_width=2)
    
    # Set up the second y-axis
    p.extra_y_ranges = {'volumes': Range1d(start=0, end=axis_range(volumes)[1])}
    axis_right = LinearAxis(y_range_name='volumes', axis_label='Trade Volume (# of ' + crypto_label + ')', formatter=NumeralTickFormatter(format="0,0a"))
    p.add_layout(axis_right, 'right')
    p.rect(times, volumes/2, width=1, height=volumes, y_range_name='volumes', color='grey', alpha=.4)
    
    return p

In [3]:
headings = ['time', 'low', 'high', 'open', 'close', 'volume']
cryptos = ['BTC', 'ETH', 'LTC', 'BCH', 'ETC']
fiats = ['USD', 'EUR']
time_slices = [60, 300, 900, 3600, 21600, 86400]

data = pd.DataFrame(get_product_data(granularity=21600), columns=headings)
data['time'] = data['time'] * 1000 # Change microseconds to milliseconds

In [4]:
fig = make_rate_plot(data['time'], data['close'], data['volume'])
show(fig)